In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

In [2]:
titanic = pd.read_csv("../metrics/titanic/cleaned.csv")
targets = titanic.Survived
data = titanic.drop(columns="Survived")
x_train, x_test, y_train, y_test = train_test_split(data, targets, train_size=0.8, random_state=0)

In [3]:
train, valid, train_true, valid_true = train_test_split(x_train, y_train, train_size=0.5, random_state=0)

In [4]:
knn = KNeighborsClassifier(n_neighbors=3)
knn_model = knn.fit(train, train_true)

lr = LogisticRegression(random_state=17)
lr_model = lr.fit(train, train_true)

dtc = DecisionTreeClassifier(max_leaf_nodes=4, random_state=17)
dtc_model = dtc.fit(train, train_true)

svc = SVC(random_state=17)
svc_model = svc.fit(train, train_true)

/home/hik0/mount/500_gb/data_science/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
models = [knn_model, lr_model, dtc_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models)))  # (num of objects, 4 algorithms)

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(x_test)
    print(f"{n} auc: {roc_auc_score(y_test, predicted)}")

meta = XGBClassifier(n_estimators=40)
meta_model = meta.fit(meta_mtrx, valid_true)

0 auc: 0.6844532279314888
1 auc: 0.7995388669301713
2 auc: 0.7877470355731225
3 auc: 0.6538866930171279


In [6]:
meta_mtrx_test = np.empty((x_test.shape[0], len(models)))

for n, model in enumerate(models):
    meta_mtrx_test[:,n] = model.predict(x_test)

meta_predict = meta.predict(meta_mtrx_test)
print(f"Blending AUC: {roc_auc_score(y_test, meta_predict)}")

Blending AUC: 0.7959815546772069


In [7]:
models = [knn_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models)))  # (num of objects, 2 algorithms)

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(x_test)
    print(f"{n} auc: {roc_auc_score(y_test, predicted)}")

meta = XGBClassifier(n_estimators=40)
meta_model = meta.fit(meta_mtrx, valid_true)

meta_mtrx_test = np.empty((x_test.shape[0], len(models)))

for n, model in enumerate(models):
    meta_mtrx_test[:,n] = model.predict(x_test)

meta_predict = meta.predict(meta_mtrx_test)
print(f"Blending AUC: {roc_auc_score(y_test, meta_predict)}")

0 auc: 0.6844532279314888
1 auc: 0.6538866930171279
Blending AUC: 0.7115942028985508
